<a href="https://colab.research.google.com/github/perses08/Sarcasm_Detect_-nlp-/blob/main/sarcasm_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

--2020-11-26 04:51:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.05s   

2020-11-26 04:51:24 (103 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [3]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras import layers as l
import json

In [4]:
with open('/tmp/sarcasm.json','r') as f:
  datastore = json.load(f)

In [6]:
sentences = []
labels = []

for i in datastore:
  sentences.append(i['headline'])
  labels.append(i['is_sarcastic'])

len(sentences)

26709

In [7]:
train_sentences = sentences[:20000]
test_sentences = sentences[20000:]

train_labels = labels[:20000]
test_labels = labels[20000:]

In [8]:
token = Tokenizer(num_words=5000, oov_token='oov')
token.fit_on_texts(sentences)

In [9]:
train_sequences = token.texts_to_sequences(train_sentences)
test_sequences = token.texts_to_sequences(test_sentences)

train = pad_sequences(train_sequences, maxlen=120, padding='post',truncating='post')
test = pad_sequences(test_sequences, maxlen=120, padding='post',truncating='post')

In [10]:
train = np.array(train)
test = np.array(test)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [11]:
model = tf.keras.Sequential([
                             l.Embedding(5000, 32, input_length=120),
                             l.Bidirectional(l.LSTM(32)),
                             l.Dense(24,activation='relu'),
                             l.Dense(1,activation='relu')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [12]:
model.fit(train, train_labels, epochs=10, validation_data=(test, test_labels), verbose=2)

Epoch 1/10
625/625 - 9s - loss: 0.5008 - accuracy: 0.7648 - val_loss: 0.4250 - val_accuracy: 0.8453
Epoch 2/10
625/625 - 8s - loss: 0.3426 - accuracy: 0.8668 - val_loss: 0.4662 - val_accuracy: 0.8457
Epoch 3/10
625/625 - 8s - loss: 0.2767 - accuracy: 0.9025 - val_loss: 0.4950 - val_accuracy: 0.8529
Epoch 4/10
625/625 - 9s - loss: 0.2338 - accuracy: 0.9250 - val_loss: 0.7802 - val_accuracy: 0.8505
Epoch 5/10
625/625 - 8s - loss: 0.2515 - accuracy: 0.9276 - val_loss: 0.9235 - val_accuracy: 0.8457
Epoch 6/10
625/625 - 8s - loss: 0.2063 - accuracy: 0.9485 - val_loss: 1.0390 - val_accuracy: 0.8416
Epoch 7/10
625/625 - 8s - loss: 0.1828 - accuracy: 0.9618 - val_loss: 1.0842 - val_accuracy: 0.8340
Epoch 8/10
625/625 - 8s - loss: 0.1654 - accuracy: 0.9713 - val_loss: 1.3572 - val_accuracy: 0.8401
Epoch 9/10
625/625 - 8s - loss: 0.1496 - accuracy: 0.9786 - val_loss: 1.5616 - val_accuracy: 0.8313
Epoch 10/10
625/625 - 8s - loss: 0.1723 - accuracy: 0.9787 - val_loss: 1.4568 - val_accuracy: 0.8328

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')